In [ ]:
!pip install vectorbt

In [ ]:
!pip install seaborn

In [1]:
import vectorbt as vbt
import pandas as pd
import talib as ta

# --- データ取得（トヨタ株） ---
price = vbt.YFData.download("7203.T", start="2021-01-01", end="2022-03-31").get("Close")
if isinstance(price, pd.DataFrame) and isinstance(price.columns, pd.MultiIndex):
    price = price.iloc[:, 0]

# --- MACD計算 ---
macd, macdsignal, _ = ta.MACD(price, fastperiod=12, slowperiod=26, signalperiod=9)

# --- エントリー/エグジット シグナル ---
entries = (macd > macdsignal).fillna(False)
exits = (macd < macdsignal).fillna(False)

# --- バックテスト実行 ---
pf = vbt.Portfolio.from_signals(price, entries, exits, init_cash=10000, fees=0.002)

# --- 結果表示 ---
print(pf.stats())
pf.plot().show()


/var/folders/gl/kj5r3kn97v70f4bpxwlhpwn40000gn/T/ipykernel_76476/3104735851.py:6: DeprecationWarning: Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.
  price = vbt.YFData.download("7203.T", start="2021-01-01", end="2022-03-31").get("Close")
/Users/cano/.pyenv/versions/3.13.2/lib/python3.13/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
/Users/cano/.pyenv/versions/3.13.2/lib/python3.13/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
/Users/cano/.pyenv/versions/3.13.2/lib/python3.13/site-packag

Start                         2021-01-03 15:00:00+00:00
End                           2022-03-29 15:00:00+00:00
Period                                              303
Start Value                                     10000.0
End Value                                  12454.284135
Total Return [%]                              24.542841
Benchmark Return [%]                          45.648866
Max Gross Exposure [%]                            100.0
Total Fees Paid                              481.112297
Max Drawdown [%]                              14.461009
Max Drawdown Duration                             116.0
Total Trades                                         12
Total Closed Trades                                  11
Total Open Trades                                     1
Open Trade PnL                              1393.484396
Win Rate [%]                                  36.363636
Best Trade [%]                                14.939432
Worst Trade [%]                               -3

In [8]:
import vectorbt as vbt
import yfinance as yf
import talib as ta
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# --- データ取得 ---
price = vbt.YFData.download('7203.T', start='2021-01-01', end='2022-03-31').get('Close')
if isinstance(price, pd.DataFrame) and isinstance(price.columns, pd.MultiIndex):
    price = price.iloc[:, 0]

# --- MACD計算 ---
macd, signal, _ = ta.MACD(price, fastperiod=12, slowperiod=26, signalperiod=9)

# --- シグナル作成 ---
entries = (macd > signal).fillna(False)
exits = (macd < signal).fillna(False)

# --- バックテスト ---
pf = vbt.Portfolio.from_signals(price, entries, exits, init_cash=10000, fees=0.002)

# --- メイントレードチャートを取得 ---
main_fig = pf.plot()

# --- MACDチャート（RSI構成に完全合わせ） ---
macd_trace = go.Scatter(x=macd.index, y=macd, name="MACD", line=dict(color="blue"))
signal_trace = go.Scatter(x=signal.index, y=signal, name="Signal", line=dict(color="red"))

macd_fig = make_subplots(rows=1, cols=1, subplot_titles=["MACD"])
macd_fig.add_trace(macd_trace)
macd_fig.add_trace(signal_trace)

macd_fig.update_yaxes(title_text="MACD")
macd_fig.update_layout(
    height=300,
    margin=dict(t=40, b=40),
    xaxis=dict(range=[price.index[0], price.index[-1]]),
)

# --- 表示 ---
main_fig.show()
macd_fig.show()


/var/folders/gl/kj5r3kn97v70f4bpxwlhpwn40000gn/T/ipykernel_76476/618122137.py:9: DeprecationWarning:

Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.



In [35]:
import vectorbt as vbt
import talib as ta
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

try:
    # データ取得
    price = vbt.YFData.download("7203.T", start="2021-01-01", end="2022-03-31", auto_adjust=True).get("Close")
    if isinstance(price, pd.DataFrame) and isinstance(price.columns, pd.MultiIndex):
        price = price.iloc[:, 0]

    # MACD計算
    macd, signal, hist = ta.MACD(price, fastperiod=12, slowperiod=26, signalperiod=9)
    macd_df = pd.DataFrame({'MACD': macd, 'Signal': signal, 'Hist': hist}, index=price.index)

    entries = (macd_df['MACD'] > macd_df['Signal']).fillna(False)
    exits = (macd_df['MACD'] < macd_df['Signal']).fillna(False)

    # バックテスト
    pf = vbt.Portfolio.from_signals(price, entries, exits, init_cash=10000, fees=0.002)

    # トレード情報（実際に売買された情報）
    trades = pf.trades.records
    # 列名の確認
    print(trades.columns)

    # 列名の修正（実際の列名に合わせてください）
    entry_dates = price.index[trades['entry_idx'].to_numpy()] #entry_idxを日付に変換
    exit_dates = price.index[trades['exit_idx'].to_numpy()] #exit_idxを日付に変換
    entry_prices = trades['entry_price'].to_numpy()
    exit_prices = trades['exit_price'].to_numpy()

    # チャート作成
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        subplot_titles=["トヨタ自動車(7203.T) 価格とトレード", "MACD"]
    )

    # 上段：価格 + トレード
    fig.add_trace(go.Scatter(x=price.index, y=price, name="Close", line=dict(color="black")), row=1, col=1)

    # トレードポイント（買いと売りのマーカー）
    fig.add_trace(go.Scatter(x=entry_dates, y=entry_prices, mode='markers',
                             marker=dict(symbol='triangle-up', color='green', size=10), name='Buy'), row=1, col=1)
    fig.add_trace(go.Scatter(x=exit_dates, y=exit_prices, mode='markers',
                             marker=dict(symbol='triangle-down', color='red', size=10), name='Sell'), row=1, col=1)

    # 下段：MACDとシグナル
    fig.add_trace(go.Scatter(x=macd_df.index, y=macd_df['MACD'], name='MACD', line=dict(color='blue')), row=2, col=1)
    fig.add_trace(go.Scatter(x=macd_df.index, y=macd_df['Signal'], name='Signal', line=dict(color='red')), row=2, col=1)
    fig.add_trace(go.Bar(x=macd_df.index, y=macd_df['Hist'], name='Histogram', marker_color='gray'), row=2, col=1)

    # レイアウト
    fig.update_layout(
        height=700,
        title="トヨタ自動車(7203.T) MACD クロス戦略",
        xaxis_title="日付",
        yaxis_title="価格",
        xaxis2_title="日付",
        yaxis2_title="MACD",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )

    fig.show()

except Exception as e:
    print(f"エラーが発生しました: {e}")

Index(['id', 'col', 'size', 'entry_idx', 'entry_price', 'entry_fees',
       'exit_idx', 'exit_price', 'exit_fees', 'pnl', 'return', 'direction',
       'status', 'parent_id'],
      dtype='object')


/var/folders/gl/kj5r3kn97v70f4bpxwlhpwn40000gn/T/ipykernel_76476/558280727.py:9: DeprecationWarning:

Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.



In [32]:
import vectorbt as vbt
import talib as ta
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

try:
    # データ取得
    price = vbt.YFData.download("7203.T", start="2021-01-01", end="2022-03-31", auto_adjust=True).get("Close")
    if isinstance(price, pd.DataFrame) and isinstance(price.columns, pd.MultiIndex):
        price = price.iloc[:, 0]

    # MACD計算
    macd, signal, hist = ta.MACD(price, fastperiod=12, slowperiod=26, signalperiod=9)
    macd_df = pd.DataFrame({'MACD': macd, 'Signal': signal, 'Hist': hist}, index=price.index)

    entries = (macd_df['MACD'] > macd_df['Signal']).fillna(False)
    exits = (macd_df['MACD'] < macd_df['Signal']).fillna(False)

    # バックテスト
    pf = vbt.Portfolio.from_signals(price, entries, exits, init_cash=10000, fees=0.002)

    # トレード情報（実際に売買された情報）
    trades = pf.trades.records
    # 列名の確認
    print(trades.columns)

    # 列名の修正（実際の列名に合わせてください）
    entry_dates = trades['Entry Timestamp'].to_numpy() if 'Entry Timestamp' in trades.columns else trades['Entry Date'].to_numpy() if 'Entry Date' in trades.columns else trades.index[trades['Entry Price'].notna()].to_numpy() # もしEntry Timestamp, Entry Dateの列が存在しない場合はindexで代用
    exit_dates = trades['Exit Timestamp'].to_numpy() if 'Exit Timestamp' in trades.columns else trades['Exit Date'].to_numpy() if 'Exit Date' in trades.columns else trades.index[trades['Exit Price'].notna()].to_numpy()# もしExit Timestamp, Exit Dateの列が存在しない場合はindexで代用
    entry_prices = trades['Entry Price'].to_numpy()
    exit_prices = trades['Exit Price'].to_numpy()

    # チャート作成
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        subplot_titles=["トヨタ自動車(7203.T) 価格とトレード", "MACD"]
    )

    # 上段：価格 + トレード
    fig.add_trace(go.Scatter(x=price.index, y=price, name="Close", line=dict(color="black")), row=1, col=1)

    # トレードポイント（買いと売りのマーカー）
    fig.add_trace(go.Scatter(x=entry_dates, y=entry_prices, mode='markers',
                             marker=dict(symbol='triangle-up', color='green', size=10), name='Buy'), row=1, col=1)
    fig.add_trace(go.Scatter(x=exit_dates, y=exit_prices, mode='markers',
                             marker=dict(symbol='triangle-down', color='red', size=10), name='Sell'), row=1, col=1)

    # 下段：MACDとシグナル
    fig.add_trace(go.Scatter(x=macd_df.index, y=macd_df['MACD'], name='MACD', line=dict(color='blue')), row=2, col=1)
    fig.add_trace(go.Scatter(x=macd_df.index, y=macd_df['Signal'], name='Signal', line=dict(color='red')), row=2, col=1)
    fig.add_trace(go.Bar(x=macd_df.index, y=macd_df['Hist'], name='Histogram', marker_color='gray'), row=2, col=1)

    # レイアウト
    fig.update_layout(
        height=700,
        title="トヨタ自動車(7203.T) MACD クロス戦略",
        xaxis_title="日付",
        yaxis_title="価格",
        xaxis2_title="日付",
        yaxis2_title="MACD",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )

    fig.show()

except Exception as e:
    print(f"エラーが発生しました: {e}")

Index(['id', 'col', 'size', 'entry_idx', 'entry_price', 'entry_fees',
       'exit_idx', 'exit_price', 'exit_fees', 'pnl', 'return', 'direction',
       'status', 'parent_id'],
      dtype='object')
エラーが発生しました: 'Entry Price'


/var/folders/gl/kj5r3kn97v70f4bpxwlhpwn40000gn/T/ipykernel_76476/326852560.py:9: DeprecationWarning:

Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.

